<a href="https://colab.research.google.com/github/mushfiqur14/MultiFace-Attendance/blob/main/Facial_Recognition_System_Using_MTCNN_and_SVM_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Face Recognition System Using FaceNet-PyTorch and MTCNN with Google Drive Integration**

**🔧 STEP 1: Install Required Libraries**

In [ ]:
!pip install mtcnn tensorflow keras-facenet scikit-learn gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.5 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=56ee5d809f6e6a4f4d362a7bf9a7bdc7b5894668de2f7cfdaf63ae69ae28662a
  Stored in directory: /root/.cache/pip/wheels/99/94/dd/cb1a65a7440ba6d508bd24346c15af0b1d24ff8b1cdb1c9959
Successfully built keras-facenet


**Install Required Libraries**

In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import pickle
import cv2
from PIL import Image, ImageDraw
import gradio as gr
from mtcnn import MTCNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras_facenet import FaceNet
from google.colab import drive

**📂 STEP 2: Mount Google Drive**

In [ ]:
drive.mount("/content/drive")
data_dir = "/content/drive/MyDrive/attendance_data"  # <-- Edit this path if needed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**🏷️ STEP 3: Load and Preprocess the Dataset**

In [ ]:
face_detector = MTCNN()
embedder = FaceNet()
embedding_list = []
labels = []

for student_name in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, student_name)
    if not os.path.isdir(person_dir):
        continue

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_detector.detect_faces(img_rgb)

        if results:
            x, y, w, h = results[0]['box']
            face = img_rgb[y:y+h, x:x+w]
            face = cv2.resize(face, (160, 160))
            embedding = embedder.embeddings(np.expand_dims(face, axis=0))[0]
            embedding_list.append(embedding)
            labels.append(student_name)

# Encode labels and train model
X = np.array(embedding_list)
le = LabelEncoder()
y = le.fit_transform(labels)

clf = SVC(probability=True, kernel='linear')
clf.fit(X, y)

# Save model
with open("/content/face_model.pkl", "wb") as f:
    pickle.dump((clf, le), f)

print("✅ Model trained successfully.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

**🧠 STEP 4: Train Face Recognition Model**

In [ ]:
# Convert embedding list to array
X = np.array(embedding_list)

# Encode the student names into numbers
le = LabelEncoder()
y = le.fit_transform(labels)

# Train SVM
clf = SVC(probability=True, kernel='linear')
clf.fit(X, y)

# Save model
with open("/content/face_model.pkl", "wb") as f:
    pickle.dump((clf, le), f)

print("✅ Model trained and saved as face_model.pkl.")

✅ Model trained and saved as face_model.pkl.


**🧪 STEP 5: Define Face Recognition Function**

In [ ]:
from PIL import Image

def recognize_faces(image_array):
    # Convert from RGB (Gradio) to BGR (OpenCV for drawing)
    img_bgr = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)

    results = face_detector.detect_faces(img_bgr)

    for result in results:
        x, y, w, h = result['box']
        x, y = max(0, x), max(0, y)
        face = img_bgr[y:y+h, x:x+w]

        if face.shape[0] < 100 or face.shape[1] < 100:
            continue

        face_resized = cv2.resize(face, (160, 160))
        embedding = embedder.embeddings(np.expand_dims(face_resized, 0))[0]
        pred = clf.predict([embedding])[0]
        name = le.inverse_transform([pred])[0]  # e.g. "213-50-073_Nishat_Morshed"

        # Draw green bounding box
        cv2.rectangle(img_bgr, (x, y), (x+w, y+h), (0, 255, 0), 3)

        # Draw medium text label above the box
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.2
        thickness = 2
        text_y = y - 10 if y - 10 > 20 else y + h + 30

        cv2.putText(
            img_bgr,
            name,
            (x, text_y),
            font,
            font_scale,
            (0, 255, 0),
            thickness,
            cv2.LINE_AA
        )

    # Convert back to RGB (for correct color display)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img_rgb)

**🌐 STEP 6: Launch Gradio Interface**

In [ ]:
iface = gr.Interface(
    fn=recognize_faces,
    inputs=gr.Image(type="numpy", label="Upload Group Photo"),
    outputs=gr.Image(type="pil", label="Labeled Result"),
    title="🎓 Face Recognition App",
    description="Upload an image to detect and identify known faces."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e78db824a21e91c00c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
